In [29]:

import pandas as pd
import re
from unidecode import unidecode
from rapidfuzz import process, fuzz

In [30]:
def normaliza_texto(texto):
    texto = unidecode(str(texto)).lower()
    texto = re.sub(r"\'s|\’s", "", texto)  # remove possessivos como "buteco's"
    texto = texto.replace('-', ' ')
    texto = re.sub(r'[^\w\s]', '', texto)  # remove pontuação restante
    return texto.strip()
def extrai_complemento(texto):
    if pd.isna(texto):
        return ''
    
    #Tenta encontrar sequência de números que identifiquem unicamente
    match_num = re.search(r'\d+', texto)
    if(match_num):
        return match_num.group(0)
    
    #Caso contrário, busca por uma letra isolada Ex: {Bloco B} -> {B}
    match_letra = re.search(r'\b([a-zA-Z])\b', texto)
    if(match_letra):
        return match_letra.group(1)
    
    #Se chegou até aqui, não deu match algum
    return ''


def encontrar_correspondencia(chave, chaves_possiveis, limiar=90):
    correspondencia, score, _ = process.extractOne(chave, chaves_possiveis, scorer=fuzz.token_sort_ratio)
    if score >= limiar:
        return correspondencia
    return None
    

In [31]:
# Carrega os arquivos
df_completo = pd.read_csv("20250401_atividade_economica_coord.csv", sep=';')
df_buteco = pd.read_csv("bares_cdb.csv")

In [32]:
# Pré-processamento do endereço
df_buteco["logradouro"] = df_buteco["Endereco"].str.split(r" \| ").str[0]
df_buteco[["rua", "numero"]] = df_buteco["logradouro"].str.extract(r"^\S+\s+(.*),\s*(\d+)")
df_buteco['comp'] = df_buteco['Complemento'].apply(extrai_complemento)

In [33]:
#Limpar os dados para facilitar no processamento
colunas_necessarias = [
    'ID_ATIV_ECON_ESTABELECIMENTO',
    'NOME_LOGRADOURO',
    'NUMERO_IMOVEL',
    'COMPLEMENTO',
    'NOME',
    'NOME_FANTASIA'
]

df_reduzido = df_completo[colunas_necessarias].copy()
df_reduzido['comp'] = df_reduzido['COMPLEMENTO'].apply(extrai_complemento)

In [34]:
#Cria as chaves para comparação

df_reduzido['chave'] = (
    df_reduzido['NOME_LOGRADOURO'].apply(normaliza_texto) + 
    '|' + 
    df_reduzido['NUMERO_IMOVEL'].astype(str) +
    '|' +
    df_reduzido['comp'].apply(normaliza_texto)
)

df_buteco['chave'] = (
    df_buteco['rua'].apply(normaliza_texto) +
    '|' +
    df_buteco['numero'].apply(normaliza_texto) +
    '|' +
    df_buteco['comp'].apply(normaliza_texto)
)

df_buteco['chave_casada'] = df_buteco['chave'].apply(
    lambda x: encontrar_correspondencia(x, df_reduzido['chave'])
)


In [ ]:
#Casa os dataframes pela chave fuzzy
df_merge = df_buteco.merge(
    df_reduzido,
    left_on='chave_casada',   
    right_on='chave',         
    how='left',
    suffixes=('_buteco', '_completo'),
    indicator=True
)

,Nome,Link Detalhes,Nome Petisco,Descricao,Endereco,Complemento,logradouro,rua,numero,comp_buteco,...,chave_casada,ID_ATIV_ECON_ESTABELECIMENTO,NOME_LOGRADOURO,NUMERO_IMOVEL,COMPLEMENTO,NOME,NOME_FANTASIA,comp_completo,chave_completo,_merge
0,Alexandre’s Bar,https://comidadibuteco.com.br/buteco/alexandre...,O fabuloso discão,Polpetone com recheio de muçarela acompanhada ...,"Rua David Alves do Vale, 68 | Santa Rosa, Belo...",Loja 01,"Rua David Alves do Vale, 68",David Alves do Vale,68,01,...,david alves do vale|68|01,22799.0,DAVID ALVES DO VALE,68.0,LOJA 01,ALEXANDRE MOREIRA DE AZEVEDO - CPF: 047.878.19...,ALEXANDRE BAR,01,david alves do vale|68|01,both
1,Amarelim do Prado,https://comidadibuteco.com.br/buteco/amarelim-...,Trem de roças,"Bochecha de porco na cama de angu mole, pesto ...","Avenida Francisco Sá, 658 | Prado, Belo Horizo...",NaN,"Avenida Francisco Sá, 658",Francisco Sá,658,,...,francisco sa|658|,324006.0,FRANCISCO SA,658.0,NaN,JANUARIO COMERCIO DE ALIMENTOS LTDA,BUTECO E LINGUICERIA AMARELIM,,francisco sa|658|,both
2,Andrade’s Beer,https://comidadibuteco.com.br/buteco/andrades-...,Chora canelinha,"Ossobuco com purê de batatas, acompanhado de t...","Rua Dona Geni, 32 | Jardim Florência (Venda No...",NaN,"Rua Dona Geni, 32",Dona Geni,32,,...,dona geni|32|,230593.0,DONA GENI,32.0,NaN,34.791.835 RONALDO VASCONCELOS DE ANDRADE,NaN,,dona geni|32|,both
3,Arcos Bar,https://comidadibuteco.com.br/buteco/arcos-bar/,Panela da Diversidade,"Acém cozido com ervas finas, acompanhado com b...","Rua da Bahia, 1144 | Centro, Belo Horizonte – MG",Loja 5,"Rua da Bahia, 1144",da Bahia,1144,5,...,da bahia|1144|5,351721.0,DA BAHIA,1144.0,"LOJA:5,",ARCOS BAR LTDA,ARCOS BAR,5,da bahia|1144|5,both
4,Arcos Bar,https://comidadibuteco.com.br/buteco/arcos-bar/,Panela da Diversidade,"Acém cozido com ervas finas, acompanhado com b...","Rua da Bahia, 1144 | Centro, Belo Horizonte – MG",Loja 5,"Rua da Bahia, 1144",da Bahia,1144,5,...,da bahia|1144|5,228827.0,DA BAHIA,1144.0,LOJA 5,ARCOS BAR E RESTAURANTE LTDA,ARCOS BAR E RESTAURANTE,5,da bahia|1144|5,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,Xico do Churrasco,https://comidadibuteco.com.br/buteco/xico-do-c...,Xico Balaio,Asinhas de frango com provolone com tempero.,"Av Altamiro avelino Soares, 920 | Castelo, Bel...",NaN,"Av Altamiro avelino Soares, 920",Altamiro avelino Soares,920,,...,altamiro avelino soares|920|,419351.0,ALTAMIRO AVELINO SOARES,920.0,NaN,CHURRASCARIA DO XICO LTDA,XICO DO CHURRASCO,,altamiro avelino soares|920|,both
137,Xico do Churrasco,https://comidadibuteco.com.br/buteco/xico-do-c...,Xico Balaio,Asinhas de frango com provolone com tempero.,"Av Altamiro avelino Soares, 920 | Castelo, Bel...",NaN,"Av Altamiro avelino Soares, 920",Altamiro avelino Soares,920,,...,altamiro avelino soares|920|,171880.0,ALTAMIRO AVELINO SOARES,920.0,NaN,MARIA BENIGNA ARAUJO FIGUEIREDO 03809132675,ER SPORT BAR,,altamiro avelino soares|920|,both
138,Xico do Churrasco,https://comidadibuteco.com.br/buteco/xico-do-c...,Xico Balaio,Asinhas de frango com provolone com tempero.,"Av Altamiro avelino Soares, 920 | Castelo, Bel...",NaN,"Av Altamiro avelino Soares, 920",Altamiro avelino Soares,920,,...,altamiro avelino soares|920|,372777.0,ALTAMIRO AVELINO SOARES,920.0,NaN,ESPACO GASTRONOMICO FOGAO CANASTRA LTDA,FOGAO CANASTRA,,altamiro avelino soares|920|,both
139,Zé Bolacha,https://comidadibuteco.com.br/buteco/ze-bolacha/,Rabicó,Bolinho de costelinha desfiada com requeijão e...,"Rua das Melancias, 35 | Vila Cloris, Belo Hori...",NaN,"Rua das Melancias, 35",das Melancias,35,,...,das melancias|35|3,102948.0,DAS MELANCIAS,35.0,LOJA 3,ZE BOLACHA LTDA,ZE BOLACHA,3,das melancias|35|3,both


In [ ]:
nomes = nomes_duplicados = df_merge['Nome'][df_merge['Nome'].duplicated()].unique()
df_repetidos = df_merge[df_merge['Nome'].isin(nomes)]


nomes_unicos = df_merge['Nome'][~df_merge['Nome'].duplicated(keep=False)]
df_match_certo = df_merge[df_merge['Nome'].isin(nomes_unicos)]
df_match_certo.to_csv("matches_quase_certos.csv", index=False, encoding="utf-8-sig")


In [43]:
df_merge = df_merge[[
    'ID_ATIV_ECON_ESTABELECIMENTO',
    'Nome',
    'NOME',
    'NOME_FANTASIA',
    'NOME_LOGRADOURO',
    'rua',
    'NUMERO_IMOVEL',
    'numero',
    '_merge'
]]
df_merge

,ID_ATIV_ECON_ESTABELECIMENTO,Nome,NOME,NOME_FANTASIA,NOME_LOGRADOURO,rua,NUMERO_IMOVEL,numero,_merge
0,22799.0,Alexandre’s Bar,ALEXANDRE MOREIRA DE AZEVEDO - CPF: 047.878.19...,ALEXANDRE BAR,DAVID ALVES DO VALE,David Alves do Vale,68.0,68,both
1,324006.0,Amarelim do Prado,JANUARIO COMERCIO DE ALIMENTOS LTDA,BUTECO E LINGUICERIA AMARELIM,FRANCISCO SA,Francisco Sá,658.0,658,both
2,230593.0,Andrade’s Beer,34.791.835 RONALDO VASCONCELOS DE ANDRADE,NaN,DONA GENI,Dona Geni,32.0,32,both
3,351721.0,Arcos Bar,ARCOS BAR LTDA,ARCOS BAR,DA BAHIA,da Bahia,1144.0,1144,both
4,228827.0,Arcos Bar,ARCOS BAR E RESTAURANTE LTDA,ARCOS BAR E RESTAURANTE,DA BAHIA,da Bahia,1144.0,1144,both
...,...,...,...,...,...,...,...,...,...
136,419351.0,Xico do Churrasco,CHURRASCARIA DO XICO LTDA,XICO DO CHURRASCO,ALTAMIRO AVELINO SOARES,Altamiro avelino Soares,920.0,920,both
137,171880.0,Xico do Churrasco,MARIA BENIGNA ARAUJO FIGUEIREDO 03809132675,ER SPORT BAR,ALTAMIRO AVELINO SOARES,Altamiro avelino Soares,920.0,920,both
138,372777.0,Xico do Churrasco,ESPACO GASTRONOMICO FOGAO CANASTRA LTDA,FOGAO CANASTRA,ALTAMIRO AVELINO SOARES,Altamiro avelino Soares,920.0,920,both
139,102948.0,Zé Bolacha,ZE BOLACHA LTDA,ZE BOLACHA,DAS MELANCIAS,das Melancias,35.0,35,both


In [38]:
df_nao_casaram = df_merge[df_merge['_merge'] == 'left_only'].copy()
df_nao_casaram

,ID_ATIV_ECON_ESTABELECIMENTO,Nome,NOME,NOME_FANTASIA,NOME_LOGRADOURO,rua,NUMERO_IMOVEL,numero,_merge
23,NaN,Bar do Romeu,NaN,NaN,NaN,Aniri,NaN,8,left_only
31,NaN,Bar Pompéu,NaN,NaN,NaN,São Miguel,NaN,779,left_only
32,NaN,Bar Stella,NaN,NaN,NaN,Saramenha,NaN,1599,left_only
36,NaN,Bazin Bar,NaN,NaN,NaN,Orozimbo Nonato,NaN,1053,left_only
37,NaN,Beco Restaurante,NaN,NaN,NaN,Tamoios,NaN,232,left_only
47,NaN,Buteco’s Bar,NaN,NaN,NaN,Ernesto Braga,NaN,2,left_only
49,NaN,Café Bahia,NaN,NaN,NaN,Tupis,NaN,369,left_only
60,NaN,Choperia América Norte Sul,NaN,NaN,NaN,Chico Rei,NaN,190,left_only
66,NaN,Deck Boi na Brasa,NaN,NaN,NaN,Desembargador José Satyro,NaN,302,left_only
72,NaN,Espetinhos do Paulão,NaN,NaN,NaN,Álvaro Mata,NaN,466,left_only


In [39]:
# 1. Separar os que não casaram ainda
df_nao_casaram = df_merge[df_merge['_merge'] == 'left_only'].copy()

# 2. Normaliza os nomes
df_nao_casaram['nome_norm'] = df_nao_casaram['Nome'].apply(normaliza_texto)
df_reduzido['nome_norm'] = df_reduzido['NOME'].apply(lambda x: normaliza_texto(x) if pd.notna(x) else '')
df_reduzido['fantasia_norm'] = df_reduzido['NOME_FANTASIA'].apply(lambda x: normaliza_texto(x) if pd.notna(x) else '')

# 3. Lista de nomes normalizados únicos
nomes_possiveis = df_reduzido['nome_norm'].dropna().unique().tolist()
fantasias_possiveis = df_reduzido['fantasia_norm'].dropna().unique().tolist()

# 4. Para cada nome do buteco não casado, tentar encontrar no df_reduzido
ids_fuzzy = []

for nome in df_nao_casaram['nome_norm']:
    match = encontrar_correspondencia(nome, nomes_possiveis)
    if match:
        linha = df_reduzido[df_reduzido['nome_norm'] == match].iloc[0]
        ids_fuzzy.append(linha['ID_ATIV_ECON_ESTABELECIMENTO'])
    else:
        match = encontrar_correspondencia(nome, fantasias_possiveis)
        if match:
            linha = df_reduzido[df_reduzido['fantasia_norm'] == match].iloc[0]
            ids_fuzzy.append(linha['ID_ATIV_ECON_ESTABELECIMENTO'])
        else:
            ids_fuzzy.append(None)

# 5. Atribui os IDs fuzzy encontrados
df_nao_casaram['ID_ATIV_ECON_ESTABELECIMENTO'] = ids_fuzzy

# 6. Seleciona apenas as colunas relevantes
df_nao_casaram = df_nao_casaram[['Nome', 'ID_ATIV_ECON_ESTABELECIMENTO']]

# 7. Atualiza o df_merge original com esses novos IDs (sem criar novas colunas)
df_merge.loc[df_merge['_merge'] == 'left_only', 'ID_ATIV_ECON_ESTABELECIMENTO'] = (
    df_merge[df_merge['_merge'] == 'left_only']
    .merge(df_nao_casaram, on='Nome', how='left')['ID_ATIV_ECON_ESTABELECIMENTO_y']
)


In [40]:
df_merge[df_merge['ID_ATIV_ECON_ESTABELECIMENTO'].isna()]

,ID_ATIV_ECON_ESTABELECIMENTO,Nome,NOME,NOME_FANTASIA,NOME_LOGRADOURO,rua,NUMERO_IMOVEL,numero,_merge
23,NaN,Bar do Romeu,NaN,NaN,NaN,Aniri,NaN,8,left_only
31,NaN,Bar Pompéu,NaN,NaN,NaN,São Miguel,NaN,779,left_only
32,NaN,Bar Stella,NaN,NaN,NaN,Saramenha,NaN,1599,left_only
36,NaN,Bazin Bar,NaN,NaN,NaN,Orozimbo Nonato,NaN,1053,left_only
37,NaN,Beco Restaurante,NaN,NaN,NaN,Tamoios,NaN,232,left_only
47,NaN,Buteco’s Bar,NaN,NaN,NaN,Ernesto Braga,NaN,2,left_only
49,NaN,Café Bahia,NaN,NaN,NaN,Tupis,NaN,369,left_only
60,NaN,Choperia América Norte Sul,NaN,NaN,NaN,Chico Rei,NaN,190,left_only
66,NaN,Deck Boi na Brasa,NaN,NaN,NaN,Desembargador José Satyro,NaN,302,left_only
72,NaN,Espetinhos do Paulão,NaN,NaN,NaN,Álvaro Mata,NaN,466,left_only
